In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mpl
import seaborn as sns

Загрузка данных

In [ ]:
train = pd.read_csv("TrainData_new.csv")
test = pd.read_csv("TestData_new.csv")

## Часть 1: Работа с пропусками


### Задание 1

Проверьте, есть ли в тренировочных и тестовых данных пропуски? Укажите количество столбцов тренировочной выборки, имеющих пропуски.

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   feature_0      906 non-null    float64
 1   feature_1      922 non-null    float64
 2   feature_2      918 non-null    float64
 3   feature_3      913 non-null    float64
 4   feature_4      904 non-null    float64
 5   feature_5      922 non-null    float64
 6   feature_6      931 non-null    float64
 7   feature_7      929 non-null    float64
 8   feature_8      914 non-null    float64
 9   feature_9      910 non-null    float64
 10  feature_10     915 non-null    float64
 11  feature_11     907 non-null    float64
 12  feature_12     920 non-null    float64
 13  feature_13     917 non-null    float64
 14  cat_feature_1  1000 non-null   object 
 15  cat_feature_2  1000 non-null   object 
 16  target         1000 non-null   int64  
dtypes: float64(14), int64(1), object(2)
memory usage: 132

In [ ]:
train.isna().sum()

feature_0        94
feature_1        78
feature_2        82
feature_3        87
feature_4        96
feature_5        78
feature_6        69
feature_7        71
feature_8        86
feature_9        90
feature_10       85
feature_11       93
feature_12       80
feature_13       83
cat_feature_1     0
cat_feature_2     0
target            0
dtype: int64

In [ ]:
test.isna().sum()

feature_0        10
feature_1        10
feature_2         8
feature_3         5
feature_4         5
feature_5         7
feature_6         8
feature_7         9
feature_8         5
feature_9         7
feature_10        8
feature_11       13
feature_12        4
feature_13        8
cat_feature_1     0
cat_feature_2     0
target            0
dtype: int64

Ответ: 14 столбцов

### Задание 2

a) В столбце с наибольшим количеством пропусков заполните пропуски средним значением по столбцу. В ответ запишите значение вычисленного среднего. Ответ округлите до десятых.

b) Найдите строки в тренировочных данных, где пропуски стоят в столбце с наименьшим количеством пропусков. Удалите эти строки. Сколько строк вы удалили?


In [ ]:
max_na_col = ""
for col in train.columns:
  if max_na_col == "" or train[max_na_col].isna().sum() < train[col].isna().sum():
    max_na_col = col

print(max_na_col, train[max_na_col].isna().sum())

feature_4 96


In [ ]:
mean = train['feature_4'].mean()
train['feature_4'].fillna(mean, inplace=True)

In [ ]:
mean

70.63761049589014

a) Ответ: 70.6

In [ ]:
min_na_col = ""
for col in train.columns:
  if train[col].isna().sum() != 0 and (min_na_col == "" or train[min_na_col].isna().sum() > train[col].isna().sum()):
    min_na_col = col

print(min_na_col, train[min_na_col].isna().sum())

feature_6 69


In [ ]:
train = train[train['feature_6'].notna()]

b) Ответ: 69

## Часть 2: Предобработка данных

### Задание 3

Выполните следующие пункты только по таблице train.

a) Сколько столбцов в таблице (не считая target) содержат меньше 5 различных значений?

b) Вычислите долю ушедших из компании клиентов, для которых значение признака 2 больше среднего значения по столбцу, а значение признака 13 меньше медианы по столбцу. Ответ округлите до сотых.


In [ ]:
train.nunique()

feature_0        843
feature_1        862
feature_2        855
feature_3        850
feature_4        841
feature_5        859
feature_6        931
feature_7        864
feature_8        849
feature_9        844
feature_10       855
feature_11       846
feature_12       853
feature_13       852
cat_feature_1      3
cat_feature_2      2
target             2
dtype: int64

a) Ответ: 2

In [ ]:
mean2 = train['feature_2'].mean()
median13 = train['feature_13'].median()

In [ ]:
subtrain = train[(train['feature_2'] > mean2) & (train['feature_13'] < median13)]

In [ ]:
len(subtrain[subtrain['target'] == 1]) / len(subtrain)

0.1111111111111111

b) Ответ: 0.11

## Часть 3: Обучение модели

### Задание 4

a) Разбейте тренировочные данные на целевой вектор y, содержащий значения из столбца target, и матрицу объект-признак X, содержащую остальные признаки. Обучите на этих данных логистическую регрессию из sklearn (LogisticRegression) с параметрами по умолчанию. Выведите среднее значение метрики f1-score алгоритма на кросс-валидации с тремя фолдами. Ответ округлите до сотых.

При объявлении модели фиксируйте random_state = 42.

Комментарий: параметры по умолчанию можете оставить дефолтными


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [ ]:
train_fit = train.copy()

In [ ]:
train_fit['cat_feature_1'].value_counts()

cat_feature_1
B    317
C    310
A    304
Name: count, dtype: int64

In [ ]:
train_fit['cat_feature_1'] = train_fit['cat_feature_1'].map({'A': 1, 'B': 2, 'C': 3})
test['cat_feature_1'] = test['cat_feature_1'].map({'A': 1, 'B': 2, 'C': 3})

In [ ]:
train_fit['cat_feature_2'].value_counts()

cat_feature_2
individuals       470
legal entities    461
Name: count, dtype: int64

In [ ]:
train_fit['individuals'] = train_fit['cat_feature_2'].map({'individuals': 1, 'legal entities': 0})
test['individuals'] = test['cat_feature_2'].map({'individuals': 1, 'legal entities': 0})

In [ ]:
train_fit.drop(columns=['cat_feature_2'], inplace=True)
test.drop(columns=['cat_feature_2'], inplace=True)

In [ ]:
# train_fit.dropna(inplace=True)

train_fit = train_fit.fillna(train_fit.mean())
test = test.fillna(test.mean())

In [ ]:
train_fit.info()

<class 'pandas.core.frame.DataFrame'>
Index: 931 entries, 0 to 999
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   feature_0      931 non-null    float64
 1   feature_1      931 non-null    float64
 2   feature_2      931 non-null    float64
 3   feature_3      931 non-null    float64
 4   feature_4      931 non-null    float64
 5   feature_5      931 non-null    float64
 6   feature_6      931 non-null    float64
 7   feature_7      931 non-null    float64
 8   feature_8      931 non-null    float64
 9   feature_9      931 non-null    float64
 10  feature_10     931 non-null    float64
 11  feature_11     931 non-null    float64
 12  feature_12     931 non-null    float64
 13  feature_13     931 non-null    float64
 14  cat_feature_1  931 non-null    int64  
 15  target         931 non-null    int64  
 16  individuals    931 non-null    int64  
dtypes: float64(14), int64(3)
memory usage: 130.9 KB


In [ ]:
X = train_fit.drop(columns=['target'])
y = train_fit['target']

In [ ]:
model = LogisticRegression()
model.fit(X, y)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
cross_val_score(model, X, y, scoring="f1", cv=3).mean()

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

0.8333166508182774

Ответ: 0.83

### Задание 5

a) Подберите значение константы регуляризации C в логистической регрессии, перебирая гиперпараметр от 0.001 до 100 включительно, проходя по степеням 10. Для выбора C примените перебор по сетке по тренировочной выборке (GridSearchCV из библиотеки sklearn.model_selection) с тремя фолдами и метрикой качества - f1-score. Остальные параметры оставьте по умолчанию. В ответ запишите наилучшее среди искомых значение C.

При объявлении модели фиксируйте random_state = 42.

Комментарий: параметры по умолчанию можете оставить дефолтными


In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'C' : [0.001, 0.01, 0.1, 1, 10, 100]}

gs = GridSearchCV(LogisticRegression(random_state = 42), params, scoring='f1', cv=3)

gs.fit(X, y)

gs.best_estimator_

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

LogisticRegression(C=0.01, random_state=42)

b) Добавьте в тренировочные и тестовые данные новый признак 'NEW', равный произведению признаков '7' и '11'. На тренировочных данных с новым признаком заново с помощью GridSearchCV (с тремя фолдами и метрикой качества - f1-score) подберите оптимальное значение C (перебирайте те же значения C, что и в предыдущих заданиях), в ответ напишите наилучшее качество алгоритма (по метрике f1-score), ответ округлите до сотых.

In [ ]:
train_fit['NEW'] = train_fit['feature_7'] * train_fit['feature_11']
test['NEW'] = test['feature_7'] * test['feature_11']

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

params = {'C' : [0.001, 0.01, 0.1, 1, 10, 100]}

gs = GridSearchCV(LogisticRegression(random_state = 42), params, scoring='f1', cv=3)

X = train_fit.drop(columns=['target'])
y = train_fit['target']

gs.fit(X, y)

gs.best_score_
gs.best_estimator_

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

LogisticRegression(C=0.1, random_state=42)

Ответ: 0.83

c) Теперь вы можете использовать любую модель машинного обучения для решения задачи. Также можете делать любую другую обработку признаков. Ваша задача - получить наилучшее качество по метрике F1-Score на тестовых данных.

In [ ]:
X_test = test.drop(columns=['target'])
y_test = test['target']

In [ ]:
pred = gs.predict(X_test)

In [ ]:
f1_score(y_test, pred)

0.760806916426513

In [ ]:
train = pd.read_csv('/content/TrainData_new.csv')
test = pd.read_csv('/content/TestData_new.csv')

In [ ]:
for col in train.columns:
  if train[col].dtype != 'object':
    train[col] = train.groupby('cat_feature_1')[col].transform(lambda x: x.fillna(x.mean()))

for col in test.columns:
  if test[col].dtype != 'object':
    test[col] = test.groupby('cat_feature_1')[col].transform(lambda x: x.fillna(x.mean()))

In [ ]:
for col in train.columns:
  if train[col].dtype != 'object' and col != 'target':
    zscore = (train[col] - train[col].mean()) / train[col].std()
    train = train[np.abs(zscore) < 3]

for col in test.columns:
  if test[col].dtype != 'object' and col != 'target':
    zscore = (test[col] - test[col].mean()) / test[col].std()
    test = test[np.abs(zscore) < 3]


In [ ]:
train['cat_feature_1'] = train['cat_feature_1'].map({'A': 1, 'B': 2, 'C': 3})
test['cat_feature_1'] = test['cat_feature_1'].map({'A': 1, 'B': 2, 'C': 3})

train['individuals'] = train['cat_feature_2'].map({'individuals': 1, 'legal entities': 0})
test['individuals'] = test['cat_feature_2'].map({'individuals': 1, 'legal entities': 0})

train.drop(columns=['cat_feature_2'], inplace=True)
test.drop(columns=['cat_feature_2'], inplace=True)

In [ ]:
train['target'].value_counts()

target
0    601
1     21
Name: count, dtype: int64

In [ ]:
df_1 = train.loc[train['target'] == 1]
df_1 = df_1.loc[df_1.index.repeat(28)]
train = pd.concat([train.loc[train['target']==0], df_1]).sample(frac=1)

In [ ]:
X_train = train.drop(columns=['target'])
y_train = train['target']

X_test = test.drop(columns=['target'])
y_test = test['target']

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

params = {'C' : [0.001, 0.01, 0.1, 1, 10, 100]}

gs = GridSearchCV(LogisticRegression(random_state = 42), params, scoring='f1', cv=3)

gs.fit(X_train, y_train)

gs.best_score_

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

0.998303647158609

In [ ]:
pred = gs_best_estimator_.predict(X_test)

In [ ]:
f1_score(y_test, pred)

0.33333333333333337

In [ ]:
pred

array([0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [ ]:
%pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.7 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier

train = pd.read_csv('/content/TrainData_new.csv')
test = pd.read_csv('/content/TestData_new.csv')

In [ ]:
mean = train['feature_4'].mean()
train['feature_4'].fillna(mean, inplace=True)

train = train[train['feature_6'].notna()]

In [ ]:
sns.heatmap(train.corr(numeric_only=True))

In [ ]:
train['target'].value_counts()

In [ ]:
train['feature_0'].hist()

In [ ]:
df_1 = train.loc[train['target'] == 1]
df_1 = df_1.loc[df_1.index.repeat(11)]
train = pd.concat([train.loc[train['target']==0], df_1]).sample(frac=1, random_state=74)

In [ ]:
# for col in train.columns:
#   if train[col].dtype != 'object' and col != 'target':
#     train['z_score'] = (train[col] - train[col].mean()) / train[col].std()
#     train = train[np.abs(train['z_score']) < 3]
#     train.drop(columns=['z_score'], inplace=True)
#     # Q1 = train[col].quantile(0.25)
#     # Q3 = train[col].quantile(0.75)
#     # IQR = Q3 - Q1
#     # train = train[(train[col] >= Q1 - 2.5 * IQR) & (train[col] <=  Q3 + 2.5 * IQR)]

In [ ]:
train['NEW'] = train['feature_0'] * train['feature_1']
test['NEW'] = test['feature_0'] * test['feature_1']

In [ ]:
train

In [ ]:
X = train.drop(columns=['target'])
y = train['target']

In [ ]:
model = CatBoostClassifier(cat_features=[14, 15])

model.fit(X, y)

Learning rate set to 0.013307
0:	learn: 0.6592078	total: 66ms	remaining: 1m 5s
1:	learn: 0.6280967	total: 82.7ms	remaining: 41.3s
2:	learn: 0.5966429	total: 98.4ms	remaining: 32.7s
3:	learn: 0.5691672	total: 113ms	remaining: 28.2s
4:	learn: 0.5469596	total: 128ms	remaining: 25.5s
5:	learn: 0.5177365	total: 142ms	remaining: 23.5s
6:	learn: 0.4933498	total: 156ms	remaining: 22.1s
7:	learn: 0.4700016	total: 175ms	remaining: 21.7s
8:	learn: 0.4486343	total: 199ms	remaining: 22s
9:	learn: 0.4265052	total: 224ms	remaining: 22.2s
10:	learn: 0.4085407	total: 239ms	remaining: 21.5s
11:	learn: 0.3910230	total: 248ms	remaining: 20.4s
12:	learn: 0.3737331	total: 256ms	remaining: 19.4s
13:	learn: 0.3571835	total: 263ms	remaining: 18.5s
14:	learn: 0.3427875	total: 270ms	remaining: 17.7s
15:	learn: 0.3267478	total: 277ms	remaining: 17s
16:	learn: 0.3115760	total: 284ms	remaining: 16.4s
17:	learn: 0.2989638	total: 291ms	remaining: 15.9s
18:	learn: 0.2873510	total: 301ms	remaining: 15.5s
19:	learn: 0.2

In [ ]:
pred = model.predict(test.drop(columns=['target']))

In [ ]:
from sklearn.metrics import f1_score

f1_score(test['target'], pred)

0.8113207547169812

In [ ]:
cross_val_score(model, X, y, scoring="f1", cv=3)

Learning rate set to 0.011192
0:	learn: 0.6651660	total: 21.6ms	remaining: 21.6s
1:	learn: 0.6391065	total: 60ms	remaining: 29.9s
2:	learn: 0.6164128	total: 78.4ms	remaining: 26s
3:	learn: 0.5898313	total: 92.1ms	remaining: 22.9s
4:	learn: 0.5703450	total: 103ms	remaining: 20.5s
5:	learn: 0.5467585	total: 110ms	remaining: 18.3s
6:	learn: 0.5267377	total: 118ms	remaining: 16.7s
7:	learn: 0.5085271	total: 131ms	remaining: 16.3s
8:	learn: 0.4911562	total: 140ms	remaining: 15.4s
9:	learn: 0.4744023	total: 146ms	remaining: 14.5s
10:	learn: 0.4580565	total: 155ms	remaining: 13.9s
11:	learn: 0.4385423	total: 165ms	remaining: 13.6s
12:	learn: 0.4206426	total: 181ms	remaining: 13.8s
13:	learn: 0.4058879	total: 200ms	remaining: 14.1s
14:	learn: 0.3906054	total: 217ms	remaining: 14.3s
15:	learn: 0.3764321	total: 230ms	remaining: 14.2s
16:	learn: 0.3620306	total: 267ms	remaining: 15.4s
17:	learn: 0.3489981	total: 278ms	remaining: 15.2s
18:	learn: 0.3350225	total: 302ms	remaining: 15.6s
19:	learn: 

array([1.       , 0.9969419, 1.       ])

In [ ]:
from sklearn.linear_model import LogisticRegression

In [4]:
def find_best_seed():
  train = pd.read_csv('/content/TrainData_new.csv')
  test = pd.read_csv('/content/TestData_new.csv')

  # mean = train['feature_4'].mean()
  # train['feature_4'].fillna(mean, inplace=True)
  # train = train[train['feature_6'].notna()]

  for col in train.columns:
    if train[col].dtype != 'object':
      train[col] = train.groupby('cat_feature_1')[col].transform(lambda x: x.fillna(x.mean()))

  for col in test.columns:
    if test[col].dtype != 'object':
      test[col] = test.groupby('cat_feature_1')[col].transform(lambda x: x.fillna(x.mean()))

  train['NEW'] = train['feature_1'] * train['feature_0']
  test['NEW'] = test['feature_1'] * test['feature_0']

  train['cat_feature_1'] = train['cat_feature_1'].map({'A': 1, 'B': 2, 'C': 3})
  test['cat_feature_1'] = test['cat_feature_1'].map({'A': 1, 'B': 2, 'C': 3})

  train['individuals'] = train['cat_feature_2'].map({'individuals': 1, 'legal entities': 0})
  test['individuals'] = test['cat_feature_2'].map({'individuals': 1, 'legal entities': 0})

  train.drop(columns=['cat_feature_2'], inplace=True)
  test.drop(columns=['cat_feature_2'], inplace=True)

  df_1 = train.loc[train['target'] == 1]
  df_1 = df_1.loc[df_1.index.repeat(10)]
  train = pd.concat([train.loc[train['target']==0], df_1]).sample(frac=1, random_state=74)

  param_grid = [
      {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
      'C' : np.logspace(-10, 10, 20),
      'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
      'max_iter' : [100, 1000,2500, 5000]
      }
  ]

  X = train.drop(columns=['target'])
  y = train['target']

  model = GridSearchCV(LogisticRegression(), param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1)
  model.fit(X,y)

  print(model.best_estimator_)

find_best_seed()

FileNotFoundError: [Errno 2] No such file or directory: '/content/TrainData_new.csv'

In [ ]:
def find_best_seed():
  train = pd.read_csv('/content/TrainData_new.csv')
  test = pd.read_csv('/content/TestData_new.csv')

  # mean = train['feature_4'].mean()
  # train['feature_4'].fillna(mean, inplace=True)
  # train = train[train['feature_6'].notna()]

  for col in train.columns:
    if train[col].dtype != 'object':
      train[col] = train.groupby('cat_feature_1')[col].transform(lambda x: x.fillna(x.mean()))

  for col in test.columns:
    if test[col].dtype != 'object':
      test[col] = test.groupby('cat_feature_1')[col].transform(lambda x: x.fillna(x.mean()))

  train['NEW'] = train['feature_1'] * train['target']
  test['NEW'] = test['feature_1'] * test['target']

  train['cat_feature_1'] = train['cat_feature_1'].map({'A': 1, 'B': 2, 'C': 3})
  test['cat_feature_1'] = test['cat_feature_1'].map({'A': 1, 'B': 2, 'C': 3})

  train['individuals'] = train['cat_feature_2'].map({'individuals': 1, 'legal entities': 0})
  test['individuals'] = test['cat_feature_2'].map({'individuals': 1, 'legal entities': 0})

  train.drop(columns=['cat_feature_2'], inplace=True)
  test.drop(columns=['cat_feature_2'], inplace=True)

  df_1 = train.loc[train['target'] == 1]
  df_1 = df_1.loc[df_1.index.repeat(10)]
  train = pd.concat([train.loc[train['target']==0], df_1]).sample(frac=1, random_state=74)

  param_grid = [
      {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
      'C' : np.logspace(-10, 10, 20),
      'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
      'max_iter' : [100, 1000,2500, 5000]
      }
  ]

  X = train.drop(columns=['target'])
  y = train['target']

  model = LogisticRegression(C=0.01)
  model.fit(X,y)

  pred = model.predict(test.drop(columns=['target']))
  print(f1_score(test['target'], pred))

find_best_seed()

0.9798270893371757


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
def find_best_seed():
  max_f1 = 0
  seed = 0
  for i in range(100):
    train = pd.read_csv('/content/TrainData_new.csv')
    test = pd.read_csv('/content/TestData_new.csv')

    mean = train['feature_4'].mean()
    train['feature_4'].fillna(mean, inplace=True)
    train = train[train['feature_6'].notna()]

    train['NEW'] = train['feature_0'] * train['feature_1']
    test['NEW'] = test['feature_0'] * test['feature_1']

    df_1 = train.loc[train['target'] == 1]
    df_1 = df_1.loc[df_1.index.repeat(10)]
    train = pd.concat([train.loc[train['target']==0], df_1]).sample(frac=1, random_state=i)

    X = train.drop(columns=['target'])
    y = train['target']

    model = CatBoostClassifier(cat_features=[14, 15])
    model.fit(X, y, silent=True)

    pred = model.predict(test.drop(columns=['target']))
    if f1_score(test['target'], pred) > max_f1:
      max_f1 = f1_score(test['target'], pred)
      seed = i
    print("cur max f1 =", max_f1, "on seed =", seed)
  print("Max f1 =", max_f1, "on seed =", seed)

find_best_seed()

cur max f1 = 0.8113207547169812 on seed = 0
cur max f1 = 0.8113207547169812 on seed = 0


KeyboardInterrupt: 

In [ ]:
from catboost import CatBoostClassifier

train = pd.read_csv('/content/TrainData_new.csv')
test = pd.read_csv('/content/TestData_new.csv')

In [ ]:
mean = train['feature_4'].mean()
train['feature_4'].fillna(mean, inplace=True)

train = train[train['feature_6'].notna()]

In [ ]:
train = train[['feature_0', 'feature_1', 'cat_feature_1', 'cat_feature_2', 'target']]

In [ ]:
train['target'].value_counts()

target
0    842
1     89
Name: count, dtype: int64

In [ ]:
df_1 = train.loc[train['target'] == 1]
df_1 = df_1.loc[df_1.index.repeat(10)]
train = pd.concat([train.loc[train['target']==0], df_1]).sample(frac=1)

In [ ]:
X = train.drop(columns=['target'])
y = train['target']

In [ ]:
model = CatBoostClassifier(cat_features=[2, 3])
model.fit(X, y)

Learning rate set to 0.013025
0:	learn: 0.6629936	total: 3.96ms	remaining: 3.96s
1:	learn: 0.6362093	total: 6.99ms	remaining: 3.49s
2:	learn: 0.6078593	total: 10.5ms	remaining: 3.5s
3:	learn: 0.5802069	total: 14.3ms	remaining: 3.55s
4:	learn: 0.5552048	total: 18.2ms	remaining: 3.61s
5:	learn: 0.5299090	total: 21.8ms	remaining: 3.61s
6:	learn: 0.5077449	total: 25.5ms	remaining: 3.62s
7:	learn: 0.4826661	total: 29.3ms	remaining: 3.63s
8:	learn: 0.4607609	total: 32.5ms	remaining: 3.58s
9:	learn: 0.4424364	total: 36.2ms	remaining: 3.58s
10:	learn: 0.4287132	total: 40ms	remaining: 3.59s
11:	learn: 0.4100233	total: 43.8ms	remaining: 3.6s
12:	learn: 0.3927966	total: 46.9ms	remaining: 3.56s
13:	learn: 0.3755333	total: 50.8ms	remaining: 3.58s
14:	learn: 0.3590218	total: 54.4ms	remaining: 3.57s
15:	learn: 0.3412984	total: 57.1ms	remaining: 3.51s
16:	learn: 0.3286646	total: 59.9ms	remaining: 3.46s
17:	learn: 0.3137469	total: 62.7ms	remaining: 3.42s
18:	learn: 0.2964938	total: 66.3ms	remaining: 3.

In [ ]:
pred = model.predict(test[['feature_0', 'feature_1', 'cat_feature_1', 'cat_feature_2']])

In [ ]:
f1_score(test['target'], pred)

0.3490566037735849